In [ ]:
import vllm
import pandas as pd

from typing import List
from datasets import load_dataset
from vllm import LLM, SamplingParams
from vllm.outputs import RequestOutput

In [ ]:
base_model= 'DopeorNope/Single_GPU_Llama3-8B'
data_id = "DopeorNope/Ko-Optimize_Dataset_train"

gpu_num= 4
max_token: int = 4096

In [ ]:
data= load_dataset(data_id)

In [ ]:
model = LLM(model=base_model, tensor_parallel_size=gpu_num, max_model_len=max_token, gpu_memory_utilization=0.6)

In [ ]:
sampling_params = SamplingParams(top_k=5, top_p=1, max_tokens=max_token,
                                     stop=['<|endoftext|>', '</s>', '<|im_end|>','<|end_of_text|>'])


In [ ]:
df= pd.DataFrame(data['train'])

In [ ]:
prompts = list(map(lambda x: 
    f"아래는 문제를 설명하는 지시사항과, 구체적인 답변 방식을 요구하는 입력이 함께 있는 문장입니다. 이 요청에 대해 적절하게 답변해주세요.\n\n###지시사항:\n{x['instruction']}\n\n###입력:\n{x['input']}\n\n###답변:\n"
    if x['input'] else
    f"아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.\n\n###지시사항:\n{x['instruction']}\n\n###답변:\n", 
    df.to_dict('records')))

In [ ]:
outputs: List[RequestOutput] = model.generate(prompts, sampling_params)

In [ ]:
outputs[0].prompt

In [ ]:
outputs[0].outputs[0].prompt

In [ ]:
extracted_prompts = list(map(lambda x: x.prompt, outputs))
extracted_generated_texts = list(map(lambda x: x.outputs[0].text, outputs))

In [ ]:
extracted_generated_texts